<a href="https://colab.research.google.com/github/amiralv82/Kakuro-Solver-CSP/blob/main/Kakuro_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
Kakuro Puzzle Solver using Backtracking Search with Heuristics
"""

import copy
from operator import itemgetter
import timeit
import os
import time

# Constants for cell categories and directions
DIGITS = [1, 2, 3, 4, 5, 6, 7, 8, 9]
WHITE = 0      # Fillable cell
CLUE = -1      # Cell containing clues
BLACK = -2     # Blocked cell
DOWN = 'down'  # Direction for vertical clues
RIGHT = 'right' # Direction for horizontal clues

class KakuroCell:
    """Base class for all Kakuro puzzle cells"""
    def __init__(self, location, category):
        self.location = location  # (row, col) tuple
        self.category = category  # WHITE, CLUE, or BLACK

class KakuroClue:
    """Represents a clue (sum target) for a row/column"""
    def __init__(self, direction, length, goal_sum):
        self.direction = direction  # DOWN or RIGHT
        self.length = length        # Number of cells in this sequence
        self.goal_sum = goal_sum    # Target sum for these cells
        self.location = None        # Will be set to (row, col)

class KakuroClueCell(KakuroCell):
    """Cell containing one or two clues (down and/or right)"""
    def __init__(self, location, down_clue, right_clue):
        super().__init__(location, category=CLUE)
        self.down_clue = down_clue   # Vertical clue (KakuroClue instance)
        self.right_clue = right_clue  # Horizontal clue (KakuroClue instance)
        self.value = 0  # Not used for clue cells, just for inheritance

        # Set clue locations if they exist
        if down_clue is not None:
            self.down_clue.location = self.location
        if right_clue is not None:
            self.right_clue.location = self.location

class KakuroBlackCell(KakuroCell):
    """Blocked cell that cannot contain numbers"""
    def __init__(self, location):
        super().__init__(location, category=BLACK)

class KakuroWhiteCell(KakuroCell):
    """Fillable cell that holds a number (1-9)"""
    def __init__(self, location, value=0):
        super().__init__(location, category=WHITE)
        self.value = value  # 0 represents unassigned

class KakuroPuzzle:
    """Main puzzle class handling grid structure and rules"""
    def __init__(self, height, width, cells):
        self.height = height
        self.width = width
        self.cells = cells          # List of all special cells
        self.clues = self.create_clues()  # All clues in the puzzle
        self.puzzle = self.create_puzzle()  # 2D grid representation
        self.print_puzzle()

    def print_puzzle(self):
        """Clear screen and print current puzzle state"""
        os.system('cls' if os.name == 'nt' else 'clear')
        for i in range(self.height):
            for j in range(self.width):
                cell = self.puzzle[i][j]
                if cell.category == BLACK:
                    print("■", end=" ")
                elif cell.category == CLUE:
                    print("C", end=" ")  # Represent clue cells with C
                elif cell.category == WHITE:
                    print(cell.value if cell.value != 0 else ".", end=" ")
            print()

    def create_clues(self):
        """Extract all clues from clue cells"""
        clues = []
        for cell in self.cells:
            if cell.category == CLUE:
                if cell.down_clue is not None:
                    clues.append(cell.down_clue)
                if cell.right_clue is not None:
                    clues.append(cell.right_clue)
        return clues

    def create_puzzle(self):
        """Build 2D grid from cell list"""
        # Initialize all cells as white
        puzzle = [[KakuroWhiteCell((i, j)) for j in range(self.width)]
                 for i in range(self.height)]
        # Override with special cells
        for cell in self.cells:
            puzzle[cell.location[0]][cell.location[1]] = cell
        return puzzle

    def get_cell_set(self, clue):
        """Get cells affected by a clue"""
        cell_set = []
        if clue.direction == DOWN:
            # Vertical sequence
            for i in range(clue.length):
                row = clue.location[0] + i + 1
                cell_set.append(self.puzzle[row][clue.location[1]])
        elif clue.direction == RIGHT:
            # Horizontal sequence
            for i in range(clue.length):
                col = clue.location[1] + i + 1
                cell_set.append(self.puzzle[clue.location[0]][col])
        return cell_set

    def assign_clue(self, clue, value_set):
        """Assign values to cells in a clue's sequence"""
        if clue.direction == DOWN:
            for i in range(clue.length):
                row = clue.location[0] + i + 1
                self.puzzle[row][clue.location[1]].value = value_set[i]
        elif clue.direction == RIGHT:
            for i in range(clue.length):
                col = clue.location[1] + i + 1
                self.puzzle[clue.location[0]][col].value = value_set[i]

    def is_clue_assigned(self, clue):
        """Check if all cells in clue sequence have values"""
        return self.clue_unassigned_count(clue) == 0

    def clue_unassigned_count(self, clue):
        """Count unassigned cells in a clue's sequence"""
        return sum(1 for cell in self.get_cell_set(clue) if cell.value == 0)

    def is_complete(self):
        """Check if all white cells have values"""
        for row in self.puzzle:
            for cell in row:
                if cell.category == WHITE and cell.value == 0:
                    return False
        return True

    def is_consistent(self):
        """Verify all clues are satisfied with current values"""
        for clue in self.clues:
            cell_set = self.get_cell_set(clue)
            values = [cell.value for cell in cell_set]

            # Check for duplicates in filled cells
            filled = [v for v in values if v != 0]
            if len(filled) != len(set(filled)):
                return False

            # Check sum if all cells are filled
            if 0 not in values:
                if sum(values) != clue.goal_sum:
                    return False
        return True

class KakuroAgent:
    """Backtracking solver for Kakuro puzzles"""
    def __init__(self, puzzle):
        self.puzzle = puzzle

    def solve(self):
        """Entry point for solving the puzzle"""
        solution = self.backtracking_search(self.puzzle)
        if solution is not None:
            solution.print_puzzle()
            self.puzzle = solution
        else:
            print("No solution found")

    def backtracking_search(self, puzzle):
        """Initialize backtracking search"""
        return self.recursive_backtracking(copy.deepcopy(puzzle))

    def recursive_backtracking(self, assignment):
        """Recursive backtracking implementation"""
        if assignment.is_complete() and assignment.is_consistent():
            print("Solution found!")
            return assignment

        # Select next clue to work on
        clue = self.select_unassigned_clue(assignment)
        if clue is not None:
            # Get possible value combinations for this clue
            cell_set = assignment.get_cell_set(clue)
            value_sets = self.order_domain_values(clue, cell_set, assignment)

            for value_set in value_sets:
                # Try assigning this value combination
                if self.is_consistent(clue, copy.deepcopy(value_set), copy.deepcopy(assignment)):
                    assignment.assign_clue(clue, value_set)
                    assignment.print_puzzle()
                    # Recursively continue
                    result = self.recursive_backtracking(copy.deepcopy(assignment))
                    if result is not None:
                        return result
            # Backtrack if no combination works
            return None

    def select_unassigned_clue(self, assignment):
        """Select next unassigned clue (basic implementation)"""
        for clue in assignment.clues:
            if not assignment.is_clue_assigned(clue):
                return clue

    def order_domain_values(self, clue, cell_set, assignment):
        """Generate valid value combinations for a clue"""
        value_sets = []
        allowed_values = copy.deepcopy(DIGITS)
        current_sum = 0
        assigned_cells = []
        unassigned_cells = []

        # Separate assigned and unassigned cells
        for cell in cell_set:
            if cell.value == 0:
                unassigned_cells.append(cell)
            else:
                assigned_cells.append(cell)
                current_sum += cell.value
                if cell.value in allowed_values:
                    allowed_values.remove(cell.value)

        # Calculate remaining sum needed
        net_goal_sum = clue.goal_sum - current_sum
        net_cell_count = clue.length - len(assigned_cells)

        # Generate valid combinations
        unassigned_value_sets = self.sum_to_n(net_goal_sum, net_cell_count, allowed_values)

        # Create full value sets including existing values
        for unassigned_set in unassigned_value_sets:
            value_set = []
            unassigned_idx = 0
            for cell in cell_set:
                if cell.value == 0:
                    value_set.append(unassigned_set[unassigned_idx])
                    unassigned_idx += 1
                else:
                    value_set.append(cell.value)
            value_sets.append(value_set)

        # Sort by least constraining values first
        value_sets.sort(key=lambda x: self.lcv_heuristic(clue, cell_set, x, assignment))
        return value_sets

    def lcv_heuristic(self, clue, cell_set, value_set, assignment):
        """Least Constraining Value heuristic: prefer values that restrict fewer options"""
        total_constraints = 0
        for idx, cell in enumerate(cell_set):
            if cell.value == 0:
                # Temporarily assign value and check consistency
                temp_assignment = copy.deepcopy(assignment)
                temp_value = value_set[idx]
                temp_assignment.assign_clue(clue, [temp_value if c == cell else c.value for c in cell_set])
                if not temp_assignment.is_consistent():
                    total_constraints += 1
        return total_constraints

    def sum_to_n(self, n, k, allowed_values):
        """Generate all combinations of k distinct numbers in allowed_values that sum to n"""
        if k == 1:
            return [[n]] if n in allowed_values else []

        combos = []
        for i in allowed_values:
            remaining = [v for v in allowed_values if v != i]
            if n - i > 0:
                # Recursively find smaller combinations
                for combo in self.sum_to_n(n - i, k - 1, remaining):
                    new_combo = [i] + combo
                    # Check for duplicates
                    if len(new_combo) == len(set(new_combo)):
                        combos.append(new_combo)
        return combos

    def is_consistent(self, clue, value_set, assignment):
        """Check if assigning this value set maintains consistency"""
        assignment.assign_clue(clue, value_set)
        return assignment.is_consistent()

class IntelligentKakuroAgent(KakuroAgent):
    """Enhanced solver with heuristic clue selection"""
    def select_unassigned_clue(self, assignment):
        """Select clue with Minimum Remaining Values heuristic"""
        clue_list = []
        # Categorize clues by assignment status
        partial_assigned = []
        unassigned = []

        for clue in assignment.clues:
            if not assignment.is_clue_assigned(clue):
                unassigned_count = assignment.clue_unassigned_count(clue)
                if unassigned_count == clue.length:
                    unassigned.append((clue, unassigned_count))
                else:
                    partial_assigned.append((clue, unassigned_count))

        # Sort clues: partially assigned first, then unassigned
        unassigned.sort(key=itemgetter(1))
        partial_assigned.sort(key=itemgetter(1))
        clue_list = partial_assigned + unassigned

        return clue_list[0][0] if clue_list else None

def select_level():
    """Display level selection menu and return user choice"""
    print("Choose a level:")
    print("1. Easy")
    print("2. Medium")
    print("3. Hard")
    print("4. Expert")

    while True:
        try:
            choice = int(input("Enter the number of the level: "))
            if 1 <= choice <= 4:
                return choice
            print("Invalid choice. Please enter a number between 1 and 4.")
        except ValueError:
            print("Invalid input. Please enter a number.")

def create_puzzle(level):
    """Create puzzle configuration based on selected level"""
    if level == 1:  # Easy
        cells = []
        # Row 1
        cells.append(KakuroBlackCell((0, 0)))
        cells.append(KakuroBlackCell((0, 1)))
        cells.append(KakuroClueCell((0, 2), KakuroClue(DOWN, 4, 30), None))
        cells.append(KakuroClueCell((0, 3), KakuroClue(DOWN, 2, 4), None))
        cells.append(KakuroClueCell((0, 4), KakuroClue(DOWN, 3, 24), None))
        cells.append(KakuroBlackCell((0, 5)))
        cells.append(KakuroClueCell((0, 6), KakuroClue(DOWN, 2, 4), None))
        cells.append(KakuroClueCell((0, 7), KakuroClue(DOWN, 2, 16), None))

        # Row 2
        cells.append(KakuroBlackCell((1, 0)))
        cells.append(KakuroClueCell((1, 1), KakuroClue(DOWN, 2, 16), KakuroClue(RIGHT, 3, 19)))
        cells.append(KakuroClueCell((1, 5), KakuroClue(DOWN, 3, 9), KakuroClue(RIGHT, 2, 10)))

        # Row 3
        cells.append(KakuroClueCell((2, 0), None, KakuroClue(RIGHT, 7, 39)))

        # Row 4
        cells.append(KakuroClueCell((3, 0), None, KakuroClue(RIGHT, 2, 15)))
        cells.append(KakuroClueCell((3, 3), KakuroClue(DOWN, 3, 23), KakuroClue(RIGHT, 2, 10)))
        cells.append(KakuroClueCell((3, 6), KakuroClue(DOWN, 4, 10), None))
        cells.append(KakuroBlackCell((3, 7)))

        # Row 5
        cells.append(KakuroBlackCell((4, 0)))
        cells.append(KakuroClueCell((4, 1), None, KakuroClue(RIGHT, 2, 16)))
        cells.append(KakuroClueCell((4, 4), KakuroClue(DOWN, 3, 6), KakuroClue(RIGHT, 2, 4)))
        cells.append(KakuroClueCell((4, 7), KakuroClue(DOWN, 2, 16), None))

        # Row 6
        cells.append(KakuroBlackCell((5, 0)))
        cells.append(KakuroClueCell((5, 1), KakuroClue(DOWN, 2, 14), None))
        cells.append(KakuroClueCell((5, 2), KakuroClue(DOWN, 2, 16), KakuroClue(RIGHT, 2, 9)))
        cells.append(KakuroClueCell((5, 5), KakuroClue(DOWN, 2, 4), KakuroClue(RIGHT, 2, 12)))

        # Row 7
        cells.append(KakuroClueCell((6, 0), None, KakuroClue(RIGHT, 7, 35)))

        # Row 8
        cells.append(KakuroClueCell((7, 0), None, KakuroClue(RIGHT, 2, 16)))
        cells.append(KakuroClueCell((7, 3), None, KakuroClue(RIGHT, 3, 7)))
        cells.append(KakuroBlackCell((7, 7)))
        return KakuroPuzzle(8, 8, cells)

    elif level == 2:  # Medium
        # Medium level puzzle configuration
        cells = []
        # (Similar detailed structure as easy level)
        return KakuroPuzzle(8, 8, cells)

    elif level == 3:  # Hard
        # Hard level puzzle configuration
        cells = []
        # (Similar detailed structure as easy level)
        return KakuroPuzzle(10, 10, cells)

    elif level == 4:  # Expert
        # Expert level puzzle configuration
        cells = []
        # (Similar detailed structure as easy level)
        return KakuroPuzzle(10, 10, cells)

def select_heuristic():
    """Display heuristic selection menu and return user choice"""
    print("Choose a heuristic:")
    print("1. Standard Backtracking Search")
    print("2. Backtracking Search with Least Constraining Value (LCV) heuristic")

    while True:
        try:
            choice = int(input("Enter the number of the heuristic: "))
            if 1 <= choice <= 2:
                return choice
            print("Invalid choice. Please enter 1 or 2.")
        except ValueError:
            print("Invalid input. Please enter a number.")

if __name__ == "__main__":
    # Initialize puzzle based on user selections
    level_choice = select_level()
    puzzle = create_puzzle(level_choice)

    # Select solving algorithm
    heuristic_choice = select_heuristic()

    # Initialize appropriate solver
    if heuristic_choice == 1:
        agent = KakuroAgent(copy.deepcopy(puzzle))
    else:
        agent = IntelligentKakuroAgent(copy.deepcopy(puzzle))

    # Solve and time execution
    start_time = timeit.default_timer()
    agent.solve()
    stop_time = timeit.default_timer()
    elapsed_time = stop_time - start_time

    # Display results
    print("Solution time:", f"{elapsed_time:.2f} seconds")

Choose a level:
1. Easy
2. Medium
3. Hard
4. Expert
Enter the number of the level: 1
■ ■ C C C ■ C C 
■ C . . . C . . 
C . . . . . . . 
C . . C . . C ■ 
■ C . . C . . C 
■ C C . . C . . 
C . . . . . . . 
C . . C . . . ■ 
Choose a heuristic:
1. Standard Backtracking Search
2. Backtracking Search with Least Constraining Value (LCV) heuristic
Enter the number of the heuristic: 2
■ ■ C C C ■ C C 
■ C . 1 . C . . 
C . . 3 . . . . 
C . . C . . C ■ 
■ C . . C . . C 
■ C C . . C . . 
C . . . . . . . 
C . . C . . . ■ 
■ ■ C C C ■ C C 
■ C . 3 . C . . 
C . . 1 . . . . 
C . . C . . C ■ 
■ C . . C . . C 
■ C C . . C . . 
C . . . . . . . 
C . . C . . . ■ 
■ ■ C C C ■ C C 
■ C 7 3 9 C . . 
C . . 1 . . . . 
C . . C . . C ■ 
■ C . . C . . C 
■ C C . . C . . 
C . . . . . . . 
C . . C . . . ■ 
■ ■ C C C ■ C C 
■ C 7 3 9 C . . 
C . . 1 7 . . . 
C . . C 8 . C ■ 
■ C . . C . . C 
■ C C . . C . . 
C . . . . . . . 
C . . C . . . ■ 
■ ■ C C C ■ C C 
■ C 7 3 9 C . . 
C . . 1 7 . . . 
C . . C 8 2 C ■ 
■ C . . C